# Inspector 1.2 to Trace Event

How to convert Inspector 1.2 format profiles to Google's Trace event format. Notebook generates Duration ('B' and 'E') events. Example file was recorded in Chrome.

In [2]:
f = open('examples/Profile-20180716T115056', 'r')

In [3]:
import json

profile = json.load(f)

In [4]:
start = None
end = None

profile_events = []
trace_events = []

for row in profile:
    if row['ph'] != 'M':
        if start is None or int(row['ts']) < start:
            start = int(row['ts'])
        if end is None or int(row['ts']) > end:
            end = int(row['ts'])
    if row['ph'] in ['B', 'E', 'X']:
        trace_events.append(row)
    elif row['ph'] == 'I' and row['name'] == 'CpuProfile':
        profile_events.append(row)

In [5]:
def parse_nodes(data):
    nodes = {}
    for node in data['cpuProfile']['nodes']:
        node_id = node['id']
        function_name = node['callFrame']['functionName']
        url = node['callFrame']['url']
        line_number = node['callFrame']['lineNumber']
        children = node.get('children')
        hit_count = node.get('hitCount')
        nodes[node_id] = {'function_name': function_name, 'url': url, 'line_number': line_number, 'hit_count': hit_count, 'children': children}
    return nodes

In [6]:
import copy

def generate_stacks(node_id, nodes, stacks, current_stack):
    node = nodes[node_id] # break in case id doesn't exist
    if node['function_name'] == '':
        node['function_name'] = '(anonymous)'
    if node['function_name'] != '(root)':
        current_stack.append(node['function_name'])
        stacks[node_id] = current_stack
    if node['children']:
        for child in node['children']:
            generate_stacks(child, nodes, stacks, copy.copy(current_stack))
    del nodes[node_id]

In [7]:
def create_begin_events(pid, tid, stack, ts):
    events = []
    # walk forward on the stack
    for frame in stack:
        events.append({
            'pid': pid,
            'tid': tid,
            'name': frame,
            'cat': 'jssample',
            'ph': 'B',
            'ts': ts
        })
    return events
    
def create_end_events(pid, tid, stack, ts):
    events = []
    # walk backwards on the stack
    for frame in reversed(stack):
        events.append({
            'pid': pid,
            'tid': tid,
            'name': frame,
            'cat': 'jssample',
            'ph': 'E',
            'ts': ts
        })
    return events

# TODO: several slices start and end at the same timestamp

def change_stack(pid, tid, previous_stack, current_stack, ts, is_last_sample):
    if is_last_sample:
        # just close everything
        return create_end_events(pid, tid, previous_stack, ts)
    stack_index = 0
    for previous_frame in previous_stack:
        current_frame = None
        if stack_index < len(current_stack):
            current_frame = current_stack[stack_index]
        if current_frame is None:
            # the previous stack is equal to the current
            # but current stack is shorter than previous
            # have to end the remaining of the previous stack
            return create_end_events(pid, tid, previous_stack[stack_index:], ts)
        if current_frame != previous_frame:
            # at this frame, the stacks differ
            # have to end the previous stack from here and begin a new stack
            events = []
            events += create_end_events(pid, tid, previous_stack[stack_index:], ts)
            events += create_begin_events(pid, tid, current_stack[stack_index:], ts)
            return events
        stack_index += 1
    if len(current_stack) > stack_index:
        # the previous stack is equal to the current
        # but the current stack is longer
        return create_begin_events(pid, tid, current_stack[stack_index:], ts) 
    # stack is the same
    return []

In [8]:
def get_events(pid, tid, samples, time_deltas, start_time, stacks, ignore_ids):
    events = []
    current_time = start_time
    previous_sample = None
    previous_stack = []
    for index, sample in enumerate(samples):
        delta = time_deltas[index]
        if delta < 0:
            delta = 0
        current_time += delta
        if sample != previous_sample:
            current_stack = stacks[sample]
            is_last_sample = index == (len(samples) - 1)
            if sample in ignore_ids:
                current_stack = []
            events += change_stack(pid, tid, previous_stack, current_stack, current_time, is_last_sample)
            previous_sample = sample
            previous_stack = current_stack
    return events

In [9]:
def get_meta_ids(nodes):
    program_node_id = None
    idle_node_id = None
    gc_node_id = None
    for key, node in nodes.items():
        if node['function_name'] == '(program)':
            program_node_id = key
        elif node['function_name'] == '(idle)':
            idle_node_id = key
        elif node['function_name'] == '(garbage collector)':
            gc_node_id = key
    return program_node_id, idle_node_id, gc_node_id

In [10]:
js_events = []

for profile in profile_events:
    pid = profile['pid']
    tid = profile['tid']
    data = profile['args']['data']
    root_id = data['cpuProfile']['nodes'][0]['id']
    nodes = parse_nodes(data)
    ignore_ids = get_meta_ids(nodes)
    stacks = {}
    generate_stacks(root_id, nodes, stacks, [])
    js_events += get_events(pid, tid, data['cpuProfile']['samples'], data['cpuProfile']['timeDeltas'], data['cpuProfile']['startTime'], stacks, ignore_ids)


In [11]:
import json

with open('inspector_1.2_trace_events.json', 'w') as file:
     file.write(json.dumps(js_events))